In [1]:
import json
import pprint
import os
from collections import defaultdict
from elasticsearch import Elasticsearch

import util

ES_CREDENTIALS = 'es-credentials.json'
RAW_DATA_FOLDER = 'data/freeze3/output'
ES_FOLDER = 'data/freeze3/elasticsearch'
QUERIES_FOLDER = 'queries/'
INDEX = 'freeze3'

In [2]:
es_client = util.get_es_client(ES_CREDENTIALS)

In [3]:
def get_indices(es_client):
    print(es_client.cat.indices(format="b", v=True))

In [4]:
get_indices(es_client)

health status index                   uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   haukur-test-bulk-4-nl   vmjsxWTiSYuHVEL62Ftqyg   1   0          0            0       261b           261b
green  open   2-analysed-2-not-en     lthgSviXTnmSg1yZrJXc7A   1   1        454            0      6.1mb            3mb
green  open   test                    sOkjwZuGTbekR7gMSXqXYA   5   1       1297            0     25.8mb         12.9mb
green  open   haukur-test-bulk-5-nl   JHt5Y4wTQRqRgG5rbXkZCw   1   0          0            0       261b           261b
green  open   exp01                   xYJmiKQSRZeS_skj8nd7uQ   1   1       1230            0       21mb         10.5mb
green  open   haukur-test-shard-1     cB6YuT4OTE6awNbhX4Iygw   1   1       5075            0     34.1mb           17mb
green  open   haukur-test-bulk-6-nl   kuO8yFZNT1eRbek_uRLodA   1   0          0            0       261b           261b
green  open   2-analysed-2-not-2-en   LTsTSpzcRc

First we check if all the documents in the queries are there

In [10]:
def are_docs_there(query_file):
    result = defaultdict(lambda: defaultdict(dict))
    with open(os.path.join(QUERIES_FOLDER, query_file), 'rt') as file:
        for query in json.load(file):
            for item in query['items']:
                result[query['queries'][0]][item['hash']]['in_raw'] = util.get_raw_document(item['hash'], RAW_DATA_FOLDER) is not None
                result[query['queries'][0]][item['hash']]['in_es'] = util.get_es_document(item['hash'], ES_FOLDER) is not None
    return result

In [11]:
pprint.pprint(are_docs_there('queries-nl.json'))

defaultdict(<function are_docs_there.<locals>.<lambda> at 0x111ff3158>,
            {'depressie': defaultdict(<class 'dict'>,
                                      {'0dd9e946f3aca97d3e7f09df68efc35a114674e7': {'in_es': False,
                                                                                    'in_raw': False},
                                       '26a14ae72d8aa1c015510d55e500fe869f426d12': {'in_es': True,
                                                                                    'in_raw': True},
                                       '4c89bc1b0f5c76b6b58fbd13e590c0921a9c42f8': {'in_es': False,
                                                                                    'in_raw': False},
                                       '7701df9213c5e144e4017c269c2f745cd9c7589d': {'in_es': False,
                                                                                    'in_raw': False},
                                       'abb17d5655caa8c1be59e9ed278a

In [12]:
pprint.pprint(are_docs_there('queries-en.json'))

defaultdict(<function are_docs_there.<locals>.<lambda> at 0x111c8e400>,
            {'biology transformation': defaultdict(<class 'dict'>,
                                                   {'2ed46d8cec0de60faa3f6066e09ad8f43befe17d': {'in_es': True,
                                                                                                 'in_raw': True},
                                                    '416218c8e4ba406507afb591b51df9ebe5a0264b': {'in_es': True,
                                                                                                 'in_raw': True},
                                                    '4dc7c22ffd942a6e4615ec3fb297f57d87c8320c': {'in_es': True,
                                                                                                 'in_raw': True},
                                                    '4e16abcb1e8588b748acf251000f88e016784851': {'in_es': True,
                                                                       

In [21]:
pprint.pprint(util.get_es_document('b56c6b39025f99ab094ea5e57e3611ffe3f1dc6b', ES_FOLDER))

{'arrangement_collection_name': 'stimuleringsregeling',
 'arrangement_keywords': ['Alteration'],
 'conformed_mime_type': 'word',
 'id': 'b56c6b39025f99ab094ea5e57e3611ffe3f1dc6b',
 'language': 'en',
 'mime_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'pipeline': {'clean': '6cbfee2'},
 'text': 'Alteration Definition of alteration Alteration in music is the '
         'changing of a basic tone, by raising or lowering it’s pitch with one '
         'or two semitones or other microtones; opening up possibilities of '
         'alteration is the first step in the direction of a chromatic system. '
         'Symbols for alteration In notation the following signs are used for '
         'this: · Sharp, to raise the pitch by one semitone; · Flat, to lower '
         'the pitch by one semitone; · Double sharp, to raise the pitch by two '
         'semitones; · Double flat, to lower the pitch by two semitones; · '
         'Natural, to correct any of the pre

In [23]:
pprint.pprint(util.get_raw_document('a000df1ed8e28fc1f9458231ac9fe60c0debe396', RAW_DATA_FOLDER))

{'arrangement_authors': ['J.T. (Johan) Jeuring'],
 'arrangement_categories': ['Educational Technology and Computing',
                            'Higher Education'],
 'arrangement_collection_name': 'figshare',
 'arrangement_keywords': ['Utrecht University',
                          'Career in education',
                          'Experiments with teaching innovations',
                          'Technology in education',
                          'Research on feedback and hints'],
 'arrangement_license': 'CC BY 4.0',
 'arrangement_summary': 'Presentation slides for the Science Education '
                        'Symposium',
 'id': 'a000df1ed8e28fc1f9458231ac9fe60c0debe396',
 'language': 'en',
 'mime_type': 'application/pdf',
 'text': '\n'
         'Sharing effective \n'
         'teaching methods\n'
         '\n'
         'Johan Jeuring\n'
         'Science Education Symposium\n'
         '\n'
         '\n'
         '\n'
         'Who am I?\n'
         '\n'
         '\n'
         '

In [ ]:
usersSearch = 'depression'
Fields = ['title^5', 'text.nl']

query = {
    'query': {
        'multi_match': { 
            'query': usersSearch, #A
            'fields': Fields, #B
        },
    },
  #  'size': '100'
}